In [78]:
# %pip install bertopic
# %pip install indicnlp-library
# %pip install transformers

In [79]:
# get processed data

import os
import pandas as pd

file_path = "../data/"

data = pd.read_csv(file_path+"train.csv")

data

,news_id,news_date,news_category,news_title,news_article
0,6,1/6/2011 2:45:49 PM,மர்மம்,தூக்கில் தொங்கு சேவல்கள் திருடர்களை காவு வாங்க...,"நாலு ஆள் உயரம், முறுக்கு மீசை, கையில் வீச்சரிவ..."
1,9,1/6/2011 2:56:51 PM,மர்மம்,பவுர்ணமி ஜாமத்தில் மாயமான கர்ப்பிணி,அமானுஷ்யமான சம்பவங்கள் நம்மை சுற்றி ஆங்காங்கே ...
2,10,1/6/2011 3:02:00 PM,இந்தியா,காமன்வெல்த் ஊழல் சுரேஷ் கல்மாடியிட கேள்விகள்,காமன்வெல்த் போட்டி ஏற்பாட்டில் நடைபெற்ற முறைகே...
3,11,1/6/2011 3:08:15 PM,மர்மம்,மச்சுபிச்சு மலை ரகசிய,தென்அமெரிக்க நாடான பெருவில் காடுகள் மிகவும் பய...
4,12,1/6/2011 3:09:20 PM,மர்மம்,ரத்த பலி வாங்கு விபரீத ஆவி,கடந்த 18ம் தேதி சாயங்காலம்... அடைமழையை கிழித்த...
...,...,...,...,...,...
9995,10152,7/14/2011 1:09:23 PM,விளையாட்டு,பயிற்சி ஆட்டத்தில் இந்தியா நாளை மோதல்,"4 டெஸ்ட் போட்டி, ஒரு 20-20, 5 ஒன்டேயில் பங்கேற..."
9996,10153,7/14/2011 1:54:26 PM,விளையாட்டு,சச்சின் புதிய அவதார நாசர் உசேன்,இங்கிலாந்து கிரிக்கெட் அணியின் மாஜி கேப்டன் நா...
9997,10154,7/14/2011 2:02:13 PM,ஸ்டேட் எக்ஸ்பிரஸ்,லோக்பாலில் பிரதமர் ராகுல்காந்தி எதிர்ப்பு,"புவனேஷ்வர்: ஊழல் அமைச்சர்கள், அதிகாரிகளை கடுமை..."
9998,10155,7/14/2011 2:10:53 PM,ஸ்டேட் எக்ஸ்பிரஸ்,நக்சல்கள் தப்பினர் போலீசார் சஸ்பெண்ட்,ராய்ப்பூர்: சட்டீஸ்கர் மாநிலம் நாராயண்பூர் மாவ...


In [80]:
# Convert to list
df=data["news_title"]

docs = [str(i) for i in df.values]

In [83]:

import torch
from transformers import BertModel, BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
model = BertModel.from_pretrained("setu4993/LaBSE")

# Set model to evaluation mode
model.eval()


"""
Encode a batch of documents using LaBSE model and return their embeddings.
Args:
    docs: List of strings representing the documents to be encoded.
    batch_size: Size of the batch to be used during encoding.
Returns:
    embeddings: Tensor of shape (n_docs, embedding_size) representing the document embeddings.
"""
# Encode the documents in batches
n_docs = len(docs)
batch_size = 8
embeds = torch.zeros((n_docs, model.config.hidden_size))
for i in range(0, n_docs, batch_size):
    batch = docs[i:i+batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    batch_embeddings = outputs.pooler_output
    embeds[i:i+batch_size] = batch_embeddings


In [84]:
#save embeddings

import pickle

with open(file_path+"embeddings.pkl", "wb") as f:
    pickle.dump(embeds, f)

#load embeddings

import pickle

with open(file_path+"embeddings.pkl", "rb") as f:

    embeds = pickle.load(f)


In [85]:
# Fine tune UMAP - DIMENSIONALITY REDUCTION STEP

from umap import UMAP


umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)



In [106]:
# Fine tune HDBSCAN - CLUSTERING STEP

from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=60, min_samples=30,
                        prediction_data=True, gen_min_span_tree=True)

In [87]:
# Tokenize docs trivially (split on spaces)

from indicnlp.tokenize import sentence_tokenize, indic_tokenize

def tokenize_ta(text,return_tensors="pt",*args,**kwargs):
    return indic_tokenize.trivial_tokenize(text)

In [88]:
# Common list of stopwords

stopwords=['அங்கு',
 'அங்கே',
 'அடுத்த',
 'அதனால்',
 'அதன்',
 'அதற்கு',
 'அதிக',
 'அதில்',
 'அது',
 'அதே',
 'அதை',
 'அந்த',
 'அந்தக்',
 'அந்தப்',
 'அன்று',
 'அல்லது',
 'அவன்',
 'அவரது',
 'அவர்',
 'அவர்கள்',
 'அவள்',
 'அவை',
 'ஆகிய',
 'ஆகியோர்',
 'ஆகும்',
 'இங்கு',
 'இங்கே',
 'இடத்தில்',
 'இடம்',
 'இதனால்',
 'இதனை',
 'இதன்',
 'இதற்கு',
 'இதில்',
 'இது',
 'இதை',
 'இந்த',
 'இந்தக்',
 'இந்தத்',
 'இந்தப்',
 'இன்னும்',
 'இப்போது',
 'இரு',
 'இருக்கும்',
 'இருந்த',
 'இருந்தது',
 'இருந்து',
 'இவர்',
 'இவை',
 'உன்',
 'உள்ள',
 'உள்ளது',
 'உள்ளன',
 'எந்த',
 'என',
 'எனக்',
 'எனக்கு',
 'எனப்படும்',
 'எனவும்',
 'எனவே',
 'எனினும்',
 'எனும்',
 'என்',
 'என்ன',
 'என்னும்',
 'என்பது',
 'என்பதை',
 'என்ற',
 'என்று',
 'என்றும்',
 'எல்லாம்',
 'ஏன்',
 'ஒரு',
 'ஒரே',
 'ஓர்',
 'கொண்ட',
 'கொண்டு',
 'கொள்ள',
 'சற்று',
 'சிறு',
 'சில',
 'சேர்ந்த',
 'தனது',
 'தன்',
 'தவிர',
 'தான்',
 'நான்',
 'நாம்',
 'நீ',
 'பற்றி',
 'பற்றிய',
 'பல',
 'பலரும்',
 'பல்வேறு',
 'பின்',
 'பின்னர்',
 'பிற',
 'பிறகு',
 'பெரும்',
 'பேர்',
 'போது',
 'போன்ற',
 'போல',
 'போல்',
 'மட்டுமே',
 'மட்டும்',
 'மற்ற',
 'மற்றும்',
 'மிக',
 'மிகவும்',
 'மீது',
 'முதல்',
 'முறை',
 'மேலும்',
 'மேல்',
 'யார்',
 'வந்த',
 'வந்து',
 'வரும்',
 'வரை',
 'வரையில்',
 'விட',
 'விட்டு',
 'வேண்டும்',
 'வேறு']

from sklearn.feature_extraction.text import CountVectorizer

# Create a vectorizer object to generate term document counts for topic representation - TOKENIZATION STEP

vectorizer_model = CountVectorizer(
    stop_words=stopwords,analyzer='word',
    tokenizer=tokenize_ta
)

In [89]:
import pandas as pd
from scipy.sparse import csr_matrix
from typing import List, Mapping, Tuple, Union
import stanza

class TamilPOS():
    """
    Extract Topic Keywords based on their Part-of-Speech using stanza library for Tamil.
    """
    def __init__(self,
                 top_n_words: int = 10,
                 pos_patterns: List[str] = None):
        self.top_n_words = top_n_words

        if pos_patterns is None:
            self.pos_patterns = [
                'NOUN',
                'PROPN',
                'ADJ',
            ]
        else:
            self.pos_patterns = pos_patterns

        # load stanza pipeline for Tamil

        self.nlp = stanza.Pipeline(lang='ta', processors='tokenize,pos')

    def extract_topics(self,
                       topic_model,
                       documents: pd.DataFrame,
                       c_tf_idf: csr_matrix,
                       topics: Mapping[str, List[Tuple[str, float]]]
                       ) -> Mapping[str, List[Tuple[str, float]]]:
        topic_to_keywords = {}
        for topic_id, topic_words in topics.items():

            # filter candidate documents that contain at least one keyword from the topic

            mask = documents['text'].str.contains('|'.join([word[0] for word in topic_words]), regex=True)
            candidate_docs = documents[mask]

            # extract candidate keywords from candidate_docs based on POS patterns

            candidate_keywords = []
            for doc in candidate_docs['text']:
                doc_keywords = []

                # get POS tags for each word in the document

                doc_words = self.nlp(doc).sentences[0].words
                for word in doc_words:
                    if word.upos in self.pos_patterns:
                        doc_keywords.append(word.text)
                candidate_keywords.extend(doc_keywords)

            # count the frequency of each keyword and keep the top n

            candidate_keyword_counts = pd.Series(candidate_keywords).value_counts().head(self.top_n_words)

            # normalize keyword counts

            candidate_keyword_counts = candidate_keyword_counts / candidate_docs.shape[0]

            # assign c-TF-IDF scores to keywords

            keyword_scores = [(word, topic_model.get_topic(topic_id)[word]) for word in candidate_keyword_counts.index]

            # sort keywords by their respective c-TF-IDF scores

            sorted_keyword_scores = sorted(keyword_scores, key=lambda x: x[1], reverse=True)

            # add top n keywords to topic_to_keywords dict
            
            topic_to_keywords[topic_id] = sorted_keyword_scores[:self.top_n_words]
        return topic_to_keywords


In [90]:
representation_model = TamilPOS()

2023-03-24 20:50:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-03-24 20:50:07 WARNING: Language ta package default expects mwt, which has been added
2023-03-24 20:50:07 INFO: Loading these models for language: ta (Tamil):
| Processor | Package |
-----------------------
| tokenize  | ttb     |
| mwt       | ttb     |
| pos       | ttb     |

2023-03-24 20:50:07 INFO: Using device: cpu
2023-03-24 20:50:07 INFO: Loading: tokenize
2023-03-24 20:50:07 INFO: Loading: mwt
2023-03-24 20:50:07 INFO: Loading: pos
2023-03-24 20:50:08 INFO: Done loading processors!


In [107]:

# Create a BERTopic model

from bertopic import BERTopic

topic_model = BERTopic(
    verbose=True,
    calculate_probabilities=False,
    embedding_model=model,
    vectorizer_model=vectorizer_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
    nr_topics=15,
    top_n_words=10,

    
)

In [108]:
# Fit the model on the documents

embeds_np = embeds.detach().numpy()
topics = topic_model.fit_transform(docs,embeds_np)

2023-03-24 21:04:08,694 - BERTopic - Reduced dimensionality
2023-03-24 21:04:08,908 - BERTopic - Clustered reduced embeddings
2023-03-24 21:04:09,818 - BERTopic - Reduced number of topics from 40 to 15


In [109]:
# Get the topics

topic_model.get_topic_info()

,Topic,Count,Name
0,-1,4728,-1_கிளிப்பிங்ஸ்_பலி_இன்று_தேர்தல்
1,0,1777,0_இந்தியா_ஜெயலலிதா_திமுக_கொலை
2,1,786,1_கருணாநிதி_முதல்வர்_ஐகோர்ட்_பார்வை
3,2,531,2_பலி_மோதி_லாரி_ரயில்
4,3,477,3_கொள்ளை_வீட்டில்_பவுன்_பிளஸ்
5,4,385,4_தேர்தல்_ஆணைய_கமிஷன்_மீனவர்கள்
6,5,325,5_நடிகை_ஹீரோயின்_இயக்குனர்_ஹீரோ
7,6,213,6_கைது_சிறை_மோசடி_ஆண்டு
8,7,157,7_சச்சின்_டோனி_புனே_சென்னை
9,8,133,8_லட்ச_கோடி_மோசடி_பண


In [110]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents

cleaned_docs = topic_model._preprocess_text(docs)

# Extract vectorizer and tokenizer from BERTopic

vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation

words = vectorizer.get_feature_names_out()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(topic_model.get_topics())-1)]

In [111]:

# Evaluate

coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence = coherence_model.get_coherence()
coherence

-12.571894924239363

In [112]:
topic_model.visualize_barchart()